In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, chi2_contingency
pd.set_option('display.max_columns',200)
pd.set_option('display.float_format', '{:,.2f}'.format)
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv('../data/raw/MachineLearningRating_v3.txt', delimiter='|', low_memory=False)

In [ ]:
df['TotalPremium'] = pd.to_numeric(df['TotalPremium'], errors='coerce')
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce')
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')

In [11]:
df['ClaimOccurred'] = df['TotalClaims'] > 0
df['ClaimSeverity'] = df['TotalClaims'] / df['ClaimOccurred'].replace({False: np.nan})
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

In [14]:
province_claims = pd.crosstab(df['Province'], df['ClaimOccurred'])
chi2, p, dof, expected = chi2_contingency(province_claims)
print("Chi-squared p-value (Claim Frequency by Province):", p)

Chi-squared p-value (Claim Frequency by Province): 5.925510718204677e-19


In [ ]:
# Convert ClaimSeverity to numeric safely
df['ClaimSeverity'] = pd.to_numeric(df['ClaimSeverity'], errors='coerce')

# Filter the top two provinces by frequency
top_provinces = df['Province'].value_counts().nlargest(2).index
grouped = df[df['Province'].isin(top_provinces)]

# Extract and convert the two groups for testing
a = pd.to_numeric(grouped[grouped['Province'] == top_provinces[0]]['ClaimSeverity'], errors='coerce').dropna()
b = pd.to_numeric(grouped[grouped['Province'] == top_provinces[1]]['ClaimSeverity'], errors='coerce').dropna()

# Now run the t-test
from scipy.stats import ttest_ind
t_stat, p_val = ttest_ind(a, b, equal_var=False)
print(f"T-test p-value (Claim Severity: {top_provinces[0]} vs {top_provinces[1]}):", p_val)

T-test p-value (Claim Severity: Gauteng vs Western Cape): 0.03059896049292002
